In [419]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

battles = pd.read_csv('data/battles_clean.csv')
conflicts = battles[['isqno', 'attacker', 'defender','winner']]
conflicts = conflicts.set_index(['isqno']) 


country1 = 'Great Britain'
conflicts = conflicts[(conflicts['attacker'] == country1) | (conflicts['defender'] == country1)]

country2 = 'USA'
conflicts = conflicts[(conflicts['attacker'] == country2) | (conflicts['defender'] == country2)]

for index, row in conflicts.iterrows():
    if row['winner'] == 'attacker':
        conflicts.at[index, 'country_winner'] = row['attacker']
    elif row['winner'] == 'defender':
        conflicts.at[index, 'country_winner'] = row['defender']
    else:
        conflicts.at[index, 'country_winner'] = 'draw'

In [420]:
conflicts

,attacker,defender,winner,country_winner
isqno,,,,
82,Great Britain,USA,attacker,Great Britain
83,USA,Great Britain,defender,Great Britain
84,Great Britain,USA,attacker,Great Britain
85,USA,Great Britain,attacker,USA
86,USA,Great Britain,attacker,USA
87,Great Britain,USA,draw,draw
88,USA,Great Britain,defender,Great Britain
89,Great Britain,USA,defender,USA
90,Great Britain,USA,draw,draw


In [421]:
conflict_count = conflicts['attacker'].value_counts().to_frame().reset_index()
conflict_count.columns = ['country', 'attacker_count']
conflict_count = conflict_count.sort_values(by='attacker_count', ascending=False)
conflict_count
attacks_count = conflict_count[conflict_count['country'] == country1]
defends_count = conflict_count[conflict_count['country'] == country2]

In [422]:
attack_data = {country2: attacks_count['attacker_count'].values[0]}
defense_data = {country2: -defends_count['attacker_count'].values[0]}

In [423]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=list(defense_data.keys()),
    x=list(defense_data.values()),
    name='Defiende',
    marker_color='blue',
    orientation='h',
    text=[-1 * x if x != 0 else None for x in defense_data.values()],
    textposition="outside"
))
fig.add_trace(go.Bar(
    y=list(attack_data.keys()),
    x=list(attack_data.values()),
    name='Ataca',
    marker_color='red',
    orientation='h',
    yaxis='y2',
    text=[x if x != 0 else None for x in attack_data.values()],
    textposition="outside"
))
fig.update_layout(
        # title=f'Conflictos que involucran a {selected_country}',
        xaxis_title='Número de batallas',
        yaxis_title='Defiende',
        yaxis2=dict(
            title='Ataca',
            overlaying='y',
            side='right'
        ),
        barmode='relative',
        margin=dict(l=20, r=20, t=20, b=20),
        legend=dict(orientation='h', yanchor='bottom',
                    y=1.02, xanchor='center', x=0.5)
    )
fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

In [424]:
wins = conflicts.groupby(['country_winner']).size().reset_index(name='wins')
wins = wins.rename(columns={'country_winner': 'country'})

In [425]:
win_data = {country1: wins[wins['country'] == country1]['wins'].values[0]}
lose_data = {country2: wins[wins['country'] == country2]['wins'].values[0]}
# si es que tiene empates (no siempre esta esta fila)
if 'draw' in wins['country'].values:
    draw_data = {'draw': wins[wins['country'] == 'draw']['wins'].values[0]}
else:
    draw_data = {'draw': 0}

In [426]:
pie_fig = go.Figure(data=[go.Pie(
    labels=['Ganadas', 'Perdidas', 'Empatadas'],
    values=[win_data[country1], lose_data[country2], draw_data['draw']],
    hole=.4,
    hoverinfo='label+percent',
    marker=dict(colors=['red', 'blue', 'gray']),
    textfont=dict(color='white'),
    hoverlabel=dict(font=dict(color='white'))
)])
# remove hover
pie_fig.update_traces(hoverinfo='skip')
pie_fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    # title=f'Porcentaje de Resultados de Batallas para {selected_country}',
    legend=dict(orientation='h', yanchor='bottom',
                y=1, xanchor='right', x=0.66),
)
pie_fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
pie_fig.show()